In [21]:
## Converte a figure to a graph
#### Author: Rafaela Scaciota <rafaela.scaciota@gmail.com>

In [1]:
# importing packages
import numpy as np
import scipy as sp
from sklearn.cluster import spectral_clustering
from sklearn.feature_extraction import image
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
import os
from os.path import join
import networkx as nx
import scipy.sparse

In [2]:
clear = lambda: os.system('cls')  # On Windows System
clear()

0

In [5]:
# Apply spectral clustering (this step goes much faster if you have pyamg
# installed)
N_REGIONS = 25
beta = 5
eps = 1e-6
p = 0.1

## Creating Graph Data and Saving

In [10]:
folder = "./data/2022-10-26_16-54-09/"
count = 0
steps = 0
maxPixel= 12
imageSize = maxPixel * maxPixel
num_features = imageSize

# Iterate directory
for path in os.listdir(folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder, path)):
        count += 1
print('File count:', count)
for f in range(count):
    print(str(steps))
    print(str(f))
    # reading image
    data = 'Frame' + str(steps) + '.png'
    path = join(folder, data)
    print(path)
    def read_image():
        img = cv2.imread(path, 0)
        return img

    cartpole = read_image()
    print ("img_processing.size=",cartpole.size)
    imageSize=maxPixel*maxPixel
    cartpole = resize(cartpole, (maxPixel, maxPixel))
    print ("img_processing.size=",cartpole.size)
    # Downsample the image by a factor of 4
    print(cartpole.shape)
    mask = cartpole.astype(bool)
    cartpole = cartpole.astype(float)
    print(cartpole.shape)

    # Convert the image into a graph with the value of the gradient on the
    # edges.
    import random

    if True:
        graph = image.img_to_graph(cartpole, mask=mask, return_as=np.ndarray)
        arr = image.img_to_graph(cartpole, mask=mask, return_as=np.ndarray)
        graph1 = image.img_to_graph(cartpole)

        G = nx.from_numpy_array(arr)
        G.edges(data=True)

        row = []
        col = []
        data = []

        for i in range(graph.shape[0]):
            for j in range(graph.shape[1]):
                if graph[i][j] != 0:
                    row.append(i)
                    col.append(j)
                    data.append(graph[i][j])

        graph = sp.sparse.coo_matrix((data, (row,col)), shape=graph.shape, dtype = float)
    else:
        graph = image.img_to_graph(cartpole)

    # Take a decreasing function of the gradient: an exponential
    # The smaller beta is, the more independent the segmentation is of the
    # actual image. For beta=1, the segmentation is close to a voronoi
    graph.data = np.exp(-beta * graph.data / cartpole.std()) + eps
    # use the tofile() method
    # and use ',' as a separator
    # as we have to generate a csv file
    arr.tofile('.\data\graph\g'+ str(steps)+'.csv', sep = ',')
    # save in graphml
    nx.write_graphml_lxml(G, '.\data\graph\g'+ str(steps)+'.graphml')
    # save in npz
    scipy.sparse.save_npz('.\data\graph\g'+ str(steps)+'.npz', graph)
    steps += 1
f += 1

File count: 176
0
0
./data/2022-10-26_16-54-09/Frame0.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
1
1
./data/2022-10-26_16-54-09/Frame1.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
2
2
./data/2022-10-26_16-54-09/Frame2.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
3
3
./data/2022-10-26_16-54-09/Frame3.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
4
4
./data/2022-10-26_16-54-09/Frame4.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
5
5
./data/2022-10-26_16-54-09/Frame5.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
6
6
./data/2022-10-26_16-54-09/Frame6.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
7
7
./data/2022-10-26_16-54-09/Frame7.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
8
8
./data/2022-10-26_16-54-09/Frame8.png
img_processing.size= 240000
img_proces

In [ ]:
print(graph.shape)
graph.data = np.exp(-graph.data / graph.data.std())
labels = spectral_clustering(graph, n_clusters = 4, eigen_solver = "arpack")

print(labels)
label_im = -np.ones(mask.shape)
label_im[mask] = labels

plt.matshow(cartpole)
plt.matshow(label_im)
plt.show()